In [1]:
import Pkg
Pkg.add("JuMP")
import Pkg; Pkg.add("JSON")
using JSON
import Pkg
Pkg.add("HiGHS")

   Resolving package versions...

   Installed MathOptInterface ─ v1.25.1


    Updating `C:\Users\caris\.julia\environments\v1.9\Project.toml`
  [4076af6c] + JuMP v1.18.1
    Updating `C:\Users\caris\.julia\environments\v1.9\Manifest.toml`


  [6e4b80f9] + BenchmarkTools v1.4.0
  [523fee87] + CodecBzip2 v0.8.1
  [bbf7d656] + CommonSubexpressions v0.3.0
  [163ba53b] + DiffResults v1.1.0
  [b552c78f] + DiffRules v1.15.1
  [f6369f11] + ForwardDiff v0.10.36
  [4076af6c] + JuMP v1.18.1
  [b8f27783] + MathOptInterface v1.25.1
  [d8a4904e] + MutableArithmetics v1.4.0
  [276daf66] + SpecialFunctions v2.3.1
  [1e83bf80] + StaticArraysCore v1.4.2
  [efe28fd5] + OpenSpecFun_jll v0.5.5+0
  [9abbd945] + Profile


Precompiling

...

   Resolving package versions...


    Updating `C:\Users\caris\.julia\environments\v1.9\Project.toml`
  [87dc4568] + HiGHS v1.7.5
    Updating `C:\Users\caris\.julia\environments\v1.9\Manifest.toml`


  [87dc4568] + HiGHS v1.7.5
  [8fd58aa0] + HiGHS_jll v1.6.0+0


Precompiling project...


  ✓ HiGHS
  1 dependency successfully precompiled in 22 seconds. 162 already precompiled.


In [2]:
# get functions

function get_parameters(name_json)
    general_parameters = name_json["general_parameters"]
    return general_parameters
end

function get_wind_turbines(name_json)
    get_wind_turbines = name_json["wind_turbines"]

    return get_wind_turbines
end

function get_wind_scenarios(name_json)
    get_wind_scenarios = name_json["wind_scenarios"]
    return get_wind_scenarios
end

function get_substation_locations(name_json)
    get_substation_locations = name_json["substation_locations"]
    
    return get_substation_locations
end

function get_substation_types(name_json)
    get_substation_types = name_json["substation_types"]

    return get_substation_types
end

function get_land_substation_cable_types(name_json)
    get_land_substation_cable_types = name_json["land_substation_cable_types"]

    return get_land_substation_cable_types
end

function get_substation_substation_cable_types(name_json)
    get_substation_substation_cable_types = name_json["substation_substation_cable_types"]

    return get_substation_substation_cable_types
end

get_substation_substation_cable_types (generic function with 1 method)

In [3]:
# data extraction

json_small = JSON.parsefile("KIRO-small.json")
file = json_small

P_max = get_parameters(file)["maximum_power"]
coords_mls = get_parameters(file)["main_land_station"]
C_ft = get_parameters(file)["fixed_cost_cable"]
C_lt = get_parameters(file)["variable_cost_cable"]
C_0 = get_parameters(file)["curtailing_cost"]
C_p = get_parameters(file)["curtailing_penalty"]
C_max = get_parameters(file)["maximum_curtailing"]
general_parameters = Dict("maximum_power" => P_max, "main_land_station" => coords_mls, "fixed_cost_cable" => C_ft, "variable_cost_cable" => C_lt, "curtailing_cost" => C_0, "curtailing_penalty" => C_p, "maximum_curtailing" => C_max)

wind_turbines = get_wind_turbines(file)
x_turbines = [turbine["x"] for turbine in wind_turbines]
y_turbines = [turbine["y"] for turbine in wind_turbines]
V_t = [x_turbines y_turbines]

wind_scenarios = get_wind_scenarios(file)

substation_locations = get_substation_locations(file)
x_substations = [substation["x"] for substation in substation_locations]
y_substations = [substation["y"] for substation in substation_locations]
V_s = [x_substations y_substations]

S = get_substation_types(file)

Q0 = get_land_substation_cable_types(file)

QS = get_substation_substation_cable_types(file)

print("done")

done

In [14]:
# Fonctions coûts 

function cost_turbines_substation(z, D1, general_parameters)
    # z : vecteur ligne indiquant la liaison entre les turbines et les sous-stations
    # D1 : matrice des distances entre les turbines et les sous-stations
    # general_parameters : paramètres généraux du problème

    # On initialise le coût
    cost = 0

    # On calcule le coût de la liaison entre les turbines et les sous-stations
    for i in 1:length(z)
        cost += z[i]*(general_parameters["fixed_cost_cable"] +D1[i]*general_parameters["variable_cost_cable"])
    end

    return cost
end

function cost_substation_substation(y, D2, substation_substation_cable_types)
    # y : matrice indiquant les arêtes où il y a une liaison entre les sous-stations, et le type de câble utilisé, idem entre les sous-stations et la station terrestre
    # S : vecteur liant les sous-stations
    # D2 : matrice des distances entre les sous-stations
    # substation_substation_cable_types : types de câbles entre les sous-stations

    # On initialise le coût
    cost = 0

    # On calcule le coût de la liaison entre les sous-stations
    for i in 1:98
        for j in 1:18
            cost += y[15 + i,j]*(substation_substation_cable_types[j]["fixed_cost"]+D2[15 + i]*substation_substation_cable_types[j]["variable_cost"])
        end
    end

    return cost
end

function cost_substation_land_substation(y, D2, land_substation_cable_types)
    # y : matrice 
    # D2 : vecteur des distances entre les sous-stations et la station terrestre
    # land_substation_cable_types : types de câbles entre les sous-stations et la station terrestre

    # On initialise le coût
    cost = 0

    # On calcule le coût de la liaison entre les sous-stations et la station terrestre
    for i in 1:15
        for j in 1:20
            cost += y[i,j]*(land_substation_cable_types[j]["fixed_cost"]+D2[i]*land_substation_cable_types[j]["variable_cost"])
        end
    end

    return cost
end

function cost_substation(x, substation_types)
    # S : vecteur liant les sous-stations
    # substation_types : types de sous-stations

    # On initialise le coût
    cost = 0

    # On calcule le coût de construction des sous-stations
    for i in 1:size(x)[1]
        for j in 1:size(x)[2]
            cost += x[i][j]*substation_types[j]["cost"]
        end
    end

    return cost
end


function c_c(C, general_parameters)
    return general_parameters["curtailing_cost"]*C + general_parameters["curtailing_penalty"]*max(0,C-general_parameters["maximum_curtailing"])
end

function C_f_aux_1(v, w, wind_scenarios, y, z, substation_substation_cable_types, land_substation_cable_types, substation_types)
    C_f_aux_1 = 0

    turbines_linked = 0
    for i in 1:75
        turbines_linked += z[i+15*v1]
    end

    C_f_aux_1 += wind_scenarios[w]["power_generation"]*turbines_linked
    
    coef = 0
    coefs = []

    if v != 1
        for i in 1:(v-1)
            push!(coefs, coef + v - i)
            coef += 15 - i
        end
    end

    for j in 1:18
        for i in 1:(15-v)
            C_f_aux_1 -= y[coef + i + 15, j]*substation_substation_cable_types[j]["rating"]
        end

    end 

    return max(0, C_f_aux_1)
end

function C_f_aux_2_aux_1(v, v1, w, wind_scenarios, z, substation_substation_cable_types, land_substation_cable_types, substation_types)
    C_f_aux_2_aux_1 = 0

    turbines_linked = 0
    for i in 1:75
        turbines_linked += z[i+15*v]
    end

    

    return min(wind_scenarios[w]["power_generation"]*turbines_linked)
end

function C_f_aux_2(v, v1, w, wind_scenarios, z, substation_substation_cable_types, land_substation_cable_types, substation_types)
    C_f_aux = 0

    turbines_linked = 0
    for i in 1:75
        turbines_linked += z[i+15*v1]
    end

    C_f_aux += wind_scenarios[w]["power_generation"]*turbines_linked

    if S[v1][2] == v
        C_f_aux += min(substation_substation_cable_types[S[v1][4]]["rating"], wind_scenarios[w]["power_generation"]*S[v][1])
    else
        C_f_aux += min(0, wind_scenarios[w]["power_generation"]*S[v][1])
    end

    if S[v1][3] != 0
        C_f_aux -= min(substation_types[S[v1][5]]["rating"],land_substation_cable_types[S[v1][3]]["rating"])
    end

    return C_f_aux
end


function C_f(v, w, wind_scenarios, S, substation_substation_cable_types, land_substation_cable_types, substation_types)
    C_fit = 0

    C_fit += C_f_aux_1 * max(0, wind_scenarios[w]["power_generation"]*S[v][1]-substation_substation_cable_types[S[v][4]]["rating"])
    

    for i in 1:length(S)
        if S[i][3] != 0
            C_fit += max(0, C_f_aux(v, i, w, wind_scenarios, S, substation_substation_cable_types, land_substation_cable_types, substation_types))
        end
    end

    return C_fit

end


function C_n(w, wind_scenarios, S, substation_substation_cable_types, land_substation_cable_types, substation_types)
    
    C_n = 0

    for i in 1:length(S)
        if S[i][3] != 0
            C_n += max(0, wind_scenarios[w]["power_generation"]*S[i][1]-min(substation_types[S[i][5]]["rating"],land_substation_cable_types[S[i][3]]["rating"]))
        end
    end

    return C_n
end
    

function p_f(v, S, land_substation_cable_types, substation_types)
    p_f = 0

    if S[v][3] != 0
        p_f += land_substation_cable_types[S[v][3]]["probability_of_failure"]

        p_f += substation_types[S[v][5]]["probability_of_failure"]
    end

    return p_f
end


function operational_cost_aux(S, w, wind_scenarios, general_parameters, substation_substation_cable_types, land_substation_cable_types, substation_types)
    cost1 = 0
    cost2 = 0

    for i in 1:length(S)
        cost1 += p_f(i, S, land_substation_cable_types, substation_types)*c_c(C_f(i, w, wind_scenarios, S, substation_substation_cable_types, land_substation_cable_types, substation_types), general_parameters)
    end

    for i in 1:length(S)
        cost2 += p_f(i, S, land_substation_cable_types, substation_types)
    end
    cost2 = (1 - cost2)*c_c(C_n(w, wind_scenarios, S, substation_substation_cable_types, land_substation_cable_types, substation_types),general_parameters)

    return cost1 + cost2
end

function operational_cost(L, S, wind_scenarios, general_parameters, substation_substation_cable_types, land_substation_cable_types, substation_types)
    # L : vecteur liant turbines et sous-stations
    # S : vecteur liant les sous-stations
    # wind_scenarios : scénarios de vent
    # general_parameters : paramètres généraux du problème

    # On initialise le coût
    cost = 0

    for w in 1:length(wind_scenarios)
        cost += wind_scenarios[w]["probability"]*operational_cost_aux(S, w, wind_scenarios, general_parameters, substation_substation_cable_types, land_substation_cable_types, substation_types)
        
    end

    return cost
end

function cost(x, y, z, D1, D2, wind_scenarios, general_parameters, substation_substation_cable_types, land_substation_cable_types, substation_types)
    # L : vecteur liant turbines et sous-stations
    # S : vecteur liant les sous-stations
    # D1 : matrice des distances entre les turbines et les sous-stations
    # D2 : matrice des distances entre les sous-stations
    # D3 : vecteur des distances entre les sous-stations et la station terrestre
    # wind_scenarios : scénarios de vent
    # general_parameters : paramètres généraux du problème
    # substation_substation_cable_types : types de câbles entre les sous-stations
    # land_substation_cable_types : types de câbles entre les sous-stations et la station terrestre
    # substation_types : types de sous-stations

    # On initialise le coût
    cost = 0

    # On calcule le coût de la liaison entre les turbines et les sous-stations
    cost += cost_turbines_substation(z, D1, general_parameters)

    # On calcule le coût de la liaison entre les sous-stations
    cost += cost_substation_substation(y, D2, substation_substation_cable_types)

    # On calcule le coût de la liaison entre les sous-stations et la station terrestre
    cost += cost_substation_land_substation(y, D2, land_substation_cable_types)

    # On calcule le coût de construction des sous-stations
    cost += cost_substation(x, substation_types)

    # On calcule le coût opérationnel
    cost += operational_cost(x, y, wind_scenarios, general_parameters, substation_substation_cable_types, land_substation_cable_types, substation_types)

    return cost
end

function c(x,y,z)
    return cost(x, y, z, D1, D2, wind_scenarios, general_parameters, substation_substation_cable_types, land_substation_cable_types, substation_types)
    
end

c (generic function with 1 method)

In [15]:
using HiGHS
using JuMP

# Create a JuMP model with HiGHS solver
model = Model(HiGHS.Optimizer)

# Define binary variables 
@variable(model, x[1:15, 1:20], binary=true)
@variable(model, y[1:113, 1:20], binary=true)
@variable(model, z[1:1125], binary=true)

# Set objective function 
@objective(model, Min, c(x,y,z))

# Constraint (1)
for v in 1:15
    @constraint(model, sum(x[v, s] for s in 1:20) <= 1)
end

# Constraint (2)
for v in 1:15
    S2 = sum(y[v, q] for q in 1:20)
    S1 = sum(x[v, s] for s in 1:20)
    @constraint(model, S1 == S2)
end

# Constraint (3)
for t in 1:75
    @constraint(model, sum(z[t+v] for v in 1:15) == 1)
end

# Constraint (4)
for v in 1:15
    S1 = sum(x[v, s] for s in 1:20)
    S2 = sum(sum(y[e, q] for q in 1:18) for e in 16:113)
    @constraint(model, S1 == S2)
end


# Solve the MILP problem
optimize!(model)


UndefVarError: UndefVarError: `D1` not defined